In [8]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats
from astroquery.vizier import Vizier
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry

In [2]:
def detect_stars(image_data, threshold_sigma=5, fwhm=3.0):
    """
    Detect stars in the FITS image using DAOStarFinder.

    Parameters:
        image_data (2D array): The FITS image data.
        threshold_sigma (float): Detection threshold in terms of image standard deviation.
        fwhm (float): Full width at half maximum for the stars in pixels.

    Returns:
        astropy.table.Table: Table of detected star positions and fluxes.
    """
    # Compute basic statistics for the image
    mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)
    
    # Detect stars
    daofind = DAOStarFinder(threshold=threshold_sigma * std, fwhm=fwhm)
    sources = daofind(image_data - median)
    
    return sources

In [3]:
def query_star_catalog(ra_center, dec_center, search_radius=0.1, catalog_name='I/345/gaia2'):
    """
    Query a star catalog (e.g., Gaia) to get reference stars near the image center.

    Parameters:
        ra_center (float): RA of the image center in degrees.
        dec_center (float): Dec of the image center in degrees.
        search_radius (float): Radius around the center to search (in degrees).
        catalog_name (str): Vizier catalog name (default: Gaia DR2).

    Returns:
        astropy.table.Table: Catalog data containing RA, Dec, and magnitudes.
    """
    vizier = Vizier(columns=['RA_ICRS', 'DE_ICRS', 'Gmag'])  # Adjust columns for your catalog
    catalog = vizier.query_region(
        SkyCoord(ra=ra_center, dec=dec_center, unit=(u.deg, u.deg), frame='icrs'),
        radius=search_radius * u.deg,
        catalog=catalog_name
    )
    return catalog[0] if len(catalog) > 0 else None

In [4]:
def match_stars_to_catalog(detected_stars, catalog, wcs, matching_radius=1.0):
    """
    Match detected stars to catalog stars using RA/Dec coordinates.

    Parameters:
        detected_stars (astropy.table.Table): Detected stars with xcentroid, ycentroid.
        catalog (astropy.table.Table): Catalog data with RA and Dec.
        wcs (astropy.wcs.WCS): WCS object to convert pixel to world coordinates.
        matching_radius (float): Matching radius in arcseconds.

    Returns:
        astropy.table.Table: Matched catalog with additional pixel coordinates.
    """
    # Convert detected stars (pixel coordinates) to RA/Dec
    detected_coords = SkyCoord.from_pixel(detected_stars['xcentroid'], detected_stars['ycentroid'], wcs)
    
    # Catalog coordinates
    catalog_coords = SkyCoord(ra=catalog['RA_ICRS'] * u.deg, dec=catalog['DE_ICRS'] * u.deg)
    
    # Match coordinates
    idx, d2d, _ = detected_coords.match_to_catalog_sky(catalog_coords)
    matches = catalog[d2d < matching_radius * u.arcsec]
    
    # Add matched pixel coordinates to the catalog
    matches['x_pixel'], matches['y_pixel'] = detected_coords[idx].to_pixel(wcs)
    return matches

In [5]:
def measure_flux(image, x, y, aperture_radius, annulus_radii):
    """
    Measure the flux of a star using aperture photometry.

    Parameters:
        image (2D array): The FITS image data.
        x, y (float): The x and y coordinates of the star in pixels.
        aperture_radius (float): Radius of the photometric aperture in pixels.
        annulus_radii (tuple): Inner and outer radii of the background annulus.

    Returns:
        float: Flux of the star with background subtracted.
    """
    # Define the aperture and annulus
    aperture = CircularAperture((x, y), r=aperture_radius)
    annulus = CircularAnnulus((x, y), r_in=annulus_radii[0], r_out=annulus_radii[1])
    
    # Create masks and calculate background
    annulus_mask = annulus.to_mask(method='center')
    annulus_data = annulus_mask.multiply(image)
    annulus_data_1d = annulus_data[annulus_data > 0]
    background_median = np.median(annulus_data_1d)
    
    # Perform aperture photometry
    phot_table = aperture_photometry(image, aperture)
    phot_table['flux'] = phot_table['aperture_sum'] - background_median * aperture.area
    
    return phot_table['flux'][0]

In [6]:
def calculate_apparent_magnitude(flux, zero_point):
    """
    Calculate the apparent magnitude from the flux and zero point.

    Parameters:
        flux (float): Measured flux of the star.
        zero_point (float): Zero-point magnitude.

    Returns:
        float: Apparent magnitude.
    """
    return -2.5 * np.log10(flux) + zero_point

def calculate_absolute_magnitude(apparent_mag, distance_pc):
    """
    Calculate the absolute magnitude of a star.

    Parameters:
        apparent_mag (float): Apparent magnitude of the star.
        distance_pc (float): Distance to the star in parsecs.

    Returns:
        float: Absolute magnitude.
    """
    return apparent_mag - 5 * np.log10(distance_pc) + 5

In [9]:
image = r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits\van_Maanen_g.fits'

In [ ]:
# Load the FITS file and WCS
hdul = fits.open(image)
image_data = hdul[0].data
wcs = WCS(hdul[0].header)

# Detect stars
detected_stars = detect_stars(image_data)

# Query catalog
catalog = query_star_catalog(ra_center=123.45, dec_center=-54.32)

# Match stars
matched_catalog = match_stars_to_catalog(detected_stars, catalog, wcs)

# Measure flux for a target star
target_flux = measure_flux(image_data, x=matched_catalog['x_pixel'][0], y=matched_catalog['y_pixel'][0], 
                           aperture_radius=5, annulus_radii=(6, 10))

# Calculate apparent and absolute magnitude
apparent_mag = calculate_apparent_magnitude(target_flux, zero_point=21.0)
absolute_mag = calculate_absolute_magnitude(apparent_mag, distance_pc=50)
print(f"Apparent Magnitude: {apparent_mag:.2f}, Absolute Magnitude: {absolute_mag:.2f}")

In [11]:
from astropy.io import fits
from astropy.wcs import WCS

# Load the FITS file and its WCS header
hdul = fits.open(image)
header = hdul[1].header
wcs = WCS(header)

# Confirm the coordinate system
print(wcs.wcs.cname)  # Should show ICRS or equivalent RA/Dec system

['', '']


Set OBSGEO-B to    28.300308 from OBSGEO-[XYZ].
Set OBSGEO-H to     2386.994 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]


In [12]:
print(wcs)

WCS Keywords

Number of WCS axes: 2
CTYPE : 'RA---TAN' 'DEC--TAN' 
CRVAL : 12.2974061364 5.37504937898 
CRPIX : 1200.5 1200.5 
CD1_1 CD1_2  : -0.00020755979988 4.50257361394e-07 
CD2_1 CD2_2  : 4.50257361394e-07 0.00020755979988 
NAXIS : 2400  2400


In [13]:
from astropy.coordinates import SkyCoord

# Example pixel coordinates of stars (replace with actual values or from detection algorithm)
x_pixels = [1189, 1329, 791]
y_pixels = [1175, 1127, 1324]

# Convert pixel coordinates to RA/Dec
ra_dec_coords = SkyCoord.from_pixel(x_pixels, y_pixels, wcs, origin=1)  # FITS uses 1-based indexing

# Print the results
for i, coord in enumerate(ra_dec_coords):
    print(f"Star {i+1}: RA = {coord.ra.deg:.6f} deg, Dec = {coord.dec.deg:.6f} deg")


Star 1: RA = 12.299792 deg, Dec = 5.369751 deg
Star 2: RA = 12.270584 deg, Dec = 5.359851 deg
Star 3: RA = 12.382837 deg, Dec = 5.400493 deg


In [14]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u

# Define a region around the center of your image
image_center = SkyCoord(ra=12.2974061364 * u.deg, dec=5.37504937898 * u.deg)  # From WCS CRVAL
search_radius = 0.2 * u.deg  # Adjust based on your image size

# Query the Gaia catalog within this region
vizier = Vizier(columns=["RA_ICRS", "DE_ICRS", "Gmag"])  # RA, Dec, and G-band magnitude
catalog = vizier.query_region(image_center, radius=search_radius, catalog="I/350/gaiaedr3")

# Extract relevant data
if len(catalog) > 0:
    gaia_stars = catalog[0]
    print(f"Found {len(gaia_stars)} stars in the Gaia catalog.")
    print(gaia_stars)
else:
    print("No stars found in the Gaia catalog within the search region.")

Found 50 stars in the Gaia catalog.
    RA_ICRS         DE_ICRS        Gmag  
      deg             deg          mag   
--------------- --------------- ---------
 12.34942483877   5.18798278683 18.340267
 12.27133085463   5.18042290545 20.805977
 12.31924606135   5.18095019658 20.758215
 12.32821766984   5.20041747013 21.820425
 12.28859044471   5.18020800815 20.931898
 12.29860030266   5.18871623287 18.021954
 12.27498376208   5.19138212416 20.751665
 12.28365589153   5.19892645169 21.147926
 12.30681649113   5.19558767037 20.207163
            ...             ...       ...
 12.23691439519   5.20002717104 18.433680
 12.24445546222   5.20429776484 20.419570
 12.21028994437   5.19500047505 21.003350
 12.22592546239   5.20943165790 20.812883
 12.22930060415   5.20941728925 17.673510
 12.20476937347   5.20853279936 14.389923
 12.20127453876   5.20844321882 20.287804
 12.27540539136   5.21379353063 21.808867
 12.24962657005   5.21245585625 22.005976
 12.24971482642   5.20334104487 20.04202

In [16]:
from astropy.coordinates import match_coordinates_sky

# Detected stars from your image
detected_stars = SkyCoord(
    ra=[12.299792, 12.270584, 12.382837] * u.deg,
    dec=[5.369751, 5.359851, 5.400493] * u.deg
)

# Gaia catalog stars (ensure proper units in degrees)
gaia_stars = SkyCoord(
    ra=catalog[0]["RA_ICRS"].data * u.deg,  # Convert the data values to degrees
    dec=catalog[0]["DE_ICRS"].data * u.deg  # Convert the data values to degrees
)

# Perform cross-match
idx, d2d, d3d = match_coordinates_sky(detected_stars, gaia_stars)

# Print results
for i, (detected, index, separation) in enumerate(zip(detected_stars, idx, d2d)):
    print(f"Star {i+1}:")
    print(f"  Detected: RA = {detected.ra.deg:.6f}, Dec = {detected.dec.deg:.6f}")
    print(f"  Closest Gaia match: RA = {gaia_stars[index].ra.deg:.6f}, Dec = {gaia_stars[index].dec.deg:.6f}")
    print(f"  Gaia Gmag = {catalog[0]['Gmag'][index]:.3f}")
    print(f"  Angular separation = {separation.arcsec:.3f} arcsec\n")


Star 1:
  Detected: RA = 12.299792, Dec = 5.369751
  Closest Gaia match: RA = 12.382756, Dec = 5.283962
  Gaia Gmag = 20.367
  Angular separation = 428.740 arcsec

Star 2:
  Detected: RA = 12.270584, Dec = 5.359851
  Closest Gaia match: RA = 12.319049, Dec = 5.237301
  Gaia Gmag = 20.905
  Angular separation = 474.152 arcsec

Star 3:
  Detected: RA = 12.382837, Dec = 5.400493
  Closest Gaia match: RA = 12.382756, Dec = 5.283962
  Gaia Gmag = 20.367
  Angular separation = 419.513 arcsec



In [21]:
from astropy.coordinates import match_coordinates_sky

# Set a maximum separation threshold (in degrees)
max_separation = 0.2 * u.deg  # 0.1 degrees

# Detected stars from your image
detected_stars = SkyCoord(
    ra=[12.299792, 12.270584, 12.382837] * u.deg,
    dec=[5.369751, 5.359851, 5.400493] * u.deg
)

# Gaia catalog stars (ensure proper units in degrees)
gaia_stars = SkyCoord(
    ra=catalog[0]["RA_ICRS"].data * u.deg,  # Convert the data values to degrees
    dec=catalog[0]["DE_ICRS"].data * u.deg  # Convert the data values to degrees
)

# Perform cross-match with separation threshold
idx, d2d, d3d = match_coordinates_sky(detected_stars, gaia_stars)

# Filter out matches with separation greater than the threshold
valid_matches = d2d < max_separation

# Print the valid matches
for i, (detected, index, separation) in enumerate(zip(detected_stars, idx, d2d)):
    if valid_matches[i]:
        print(f"Star {i+1}:")
        print(f"  Detected: RA = {detected.ra.deg:.6f}, Dec = {detected.dec.deg:.6f}")
        print(f"  Closest Gaia match: RA = {gaia_stars[index].ra.deg:.6f}, Dec = {gaia_stars[index].dec.deg:.6f}")
        print(f"  Gaia Gmag = {catalog[0]['Gmag'][index]:.3f}")
        print(f"  Angular separation = {separation.arcsec:.3f} arcsec\n")
    else:
        # Access the value and unit separately for max_separation
        print(f"Star {i+1}: No valid match found within {max_separation.value} {max_separation.unit}.")


Star 1:
  Detected: RA = 12.299792, Dec = 5.369751
  Closest Gaia match: RA = 12.382756, Dec = 5.283962
  Gaia Gmag = 20.367
  Angular separation = 428.740 arcsec

Star 2:
  Detected: RA = 12.270584, Dec = 5.359851
  Closest Gaia match: RA = 12.319049, Dec = 5.237301
  Gaia Gmag = 20.905
  Angular separation = 474.152 arcsec

Star 3:
  Detected: RA = 12.382837, Dec = 5.400493
  Closest Gaia match: RA = 12.382756, Dec = 5.283962
  Gaia Gmag = 20.367
  Angular separation = 419.513 arcsec



In [22]:
from analysis_functions import *

In [23]:
calculate_flux(r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits', 'van_Maanen_', 1189, 1175, 8)

([171424.86, 122294.5, 51023.08],
 [21.65063765247328, 21.025287319733366, 20.03855919309204])

In [24]:
apparent_mag(171424.86, 21.65063765247328)

8.56545314383315

In [25]:
relative_to_absolute_magnitude(8.56545314383315, 4.31)

10.393066793029492

In [26]:
def apparent_mag2(flux, zeropoint, exptime):
    """Calculate the apparent magnitude."""
    # return -2.5 * np.log10(flux) + zeropoint
    return -2.5 * np.log10(flux / exptime) + zeropoint

In [27]:
apparent_mag2(171424.86, 21.65063765247328, 40)

12.570603122153056

In [28]:
relative_to_absolute_magnitude(12.570603122153056, 4.31)

14.398216771349398

In [ ]:
# gain: 0.7 e-/ADU
# Flux = Gain * Counts / Exptime.
# m = -2.5 * log10(F) + zeropoint

In [ ]:
F = 51023*0.7/40
print(F)
zp = 20.03855919309204
d = 4.3
m = -2.5 * np.log10(F) + zp
print(m)
M = m - 5 * np.log10(d) + 5
print(M)

892.9024999999999
12.661549095769193
14.489162744965535


In [35]:
m-5*np.log10(4.31)+5

14.785477118221971

In [63]:
calculate_flux(r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits', 'LAWD_8_', 1208, 1216, 8)

([944.6901406249999, 551.456390625, 167.00490624999998],
 [21.62101808246613, 21.085191937037823, 19.931519508237372],
 [67477.87, 39389.742, 11928.922])

In [64]:
F = 11928*0.7/50
print(F)
zp = 19.931519508237372
d = 16
m = -2.5 * np.log10(F) + zp
print(m)
M = m - 5 * np.log10(d) + 5
print(M)

166.99200000000002
14.374780342929432
13.354180429649809


In [62]:
def calculate_flux(file_root, star_name, x_position, y_position, radius):
    """Calculate flux within an aperture of (radius) pixels, then subtract the median background counts. 
    star_name is a string that is the start of the file name (excluding the filter, like 'van_Maanen_')"""

    gain = 0.7 # e-/ADU
    pixel_sums = []
    fluxes = []
    filter_zeropoints = []

    for filter in filters:
        file = os.path.join(file_root, star_name + f"{filter}" + '.fits')

        hdul = fits.open(file)
        data = hdul[1].data
        header = hdul[1].header
        zeropoint = header.get('L1ZP', 'Keyword not found')
        exptime = header.get('EXPTIME', 'Keyword not found')
        filter_zeropoints.append(zeropoint)

        y, x = np.indices(data.shape)  # Create coordinate grids for the image
        distance = np.sqrt((x - x_position)**2 + (y - y_position)**2)  # Distance of each pixel from the star center

        aperture_mask = distance <= radius

        background_mask = (distance > radius + 1) & (distance <= radius + 10)
        background_values = data[background_mask]  # Pixels outside the aperture
        background_median = np.median(background_values)  # Median background level

        aperture_flux = np.sum(data[aperture_mask] - background_median)

        pixel_sums.append(aperture_flux)
        flux = gain*aperture_flux/exptime
        fluxes.append(flux)

    return fluxes, filter_zeropoints, pixel_sums